# AlphaZero implementation for pulse sequence design
_Will Kaufman, December 2020_

[Dalgaard et. al. (2020)](https://www.nature.com/articles/s41534-019-0241-0) applied this approach to constructing shaped pulses (as I understand it), but in theory this should be as applicable to pulse sequence design, if not more so. The original [AlphaZero paper](https://science.sciencemag.org/content/362/6419/1140.full) is here.

The general idea behind AlphaZero (as I understand it) is to do a "smart" tree search that balances previous knowledge (the policy), curiosity in unexplored branches, and high-value branches. My thought is that this can be improved with AHT (i.e. knowing that by the end of the pulse sequence, the pulse sequence must be cyclic (the overall frame transformation must be identity) and there must be equal times spent on each axis). This will provide a hard constraint that will (hopefully) speed up search.

In [ ]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
import sys, os

In [ ]:
sys.path.append(os.path.abspath('..'))
import pulse_sequences as ps
import alpha_zero as az

In [ ]:
import importlib
importlib.reload(az)
importlib.reload(ps)

## Define the spin system

In [ ]:
delay = 1e-2  # time is relative to chemical shift strength
pulse_width = 5e-3
N = 3  # number of spins
ensemble_size = 5

In [ ]:
X, Y, Z = ps.get_collective_spin(N)

In [ ]:
Hsys_ensemble = [ps.get_Hsys(N) for _ in range(ensemble_size)]
pulses_ensemble = [
    ps.get_pulses(H, X, Y, Z, pulse_width, delay, rot_error=0.01) for H in Hsys_ensemble
]

In [ ]:
Utarget = qt.identity(Hsys_ensemble[0].dims[0])

## Average Hamiltonian theory

To keep track of the average Hamiltonian (to lowest order), I'm defining a frame matrix and applying rotation matrices to the frame matrix, then determining how $I_z$ transforms during the pulse sequence. The last row in the frame matrix corresponds to the current transformed value of $I_z$.

In [ ]:
ps.count_axes(ps.yxx48)

In [ ]:
ps.get_valid_time_suspension_pulses([0,1,1,], len(ps.pulse_names), 6)

## Tree search

Define nodes that can be used for tree search, with additional constraints that the lowest-order average Hamiltonian matches the desired Hamiltonian.

(deleted code that implemented tree search with constraints, see GitHub repo commits on 12/8 for code)

For 12-pulse sequences, calculated 16 branches at depth 4 in a minute, so about 1 every 4 seconds. At depth 4 there are $5^4 = 625$ branches, so that'll take $4 * 625 = 41$ hours to fully run. Alternatively, you can generate random pulse sequences until there's one that has the proper lowest-order average and cyclic property.

## Smarter search with MCTS

Following the [supplementary materials description under "Search"](https://science.sciencemag.org/content/sci/suppl/2018/12/05/362.6419.1140.DC1/aar6404-Silver-SM.pdf) to do rollouts and backpropagate information.

In [ ]:
config = az.Config()

In [ ]:
ps_config = ps.PulseSequenceConfig(N, ensemble_size, pulse_width, delay, 6, Utarget)

In [ ]:
a, b = az.run_mcts(config, ps_config, None)

In [ ]:
a

In [ ]:
b.max_value

In [ ]:
[c.max_value for c in b.children.values()]

In [ ]:
pulse, root = run_mcts(config, [Utarget] * 5, [], 6, Utarget)

In [ ]:
max(node.children.values(), key=lambda x: x.max_value)

In [ ]:
max([(1,'a'), (2, 'b'), (3, 'c')])

In [ ]:
node = root
while node.has_children():
    node = max(node.children.values(), key=lambda x: x.max_value)

In [ ]:
node.sequence

In [ ]:
node.max_value

In [ ]:
[root.children[p].visit_count for p in root.children]

In [ ]:
root.children[3].value()

In [ ]:
def make_sequence(config, sequence_length, network=None):
    """Start with no pulses, do MCTS until a sequence of length
    sequence_length is made.
    """
    sequence = []
    propagators = [Utarget] * 5
    search_statistics = []
    while len(sequence) < sequence_length:
        pulse, root = run_mcts(config, propagators, sequence, sequence_length, Utarget)
        print(f'applying pulse {pulse}')
        sequence.append(pulse)
        propagators = root.children[pulse].propagators
        search_statistics.append(
            (root.sequence,
             [(p, root.children[p].visit_count) for p in root.children])
        )
    return sequence, search_statistics

In [ ]:
config.num_simulations = 1000

In [ ]:
sequence, search_statistics = make_sequence(config, 24)

In [ ]:
sequence